# Predicting Housing Prices using Tensorflow + Cloud ML Engine

This notebook will show you how to create a tensorflow model, train it on the cloud in a distributed fashion across multiple CPUs or GPUs, explore the results using Tensorboard, and finally deploy the model for online prediction. We will demonstrate this by building a model to predict housing prices.

**This notebook is intended to be run on Google Cloud Datalab**: https://cloud.google.com/datalab/docs/quickstarts

Datalab will have the required libraries installed by default for this code to work. If you choose to run this code outside of Datalab you may run in to version and dependency issues which you will need to resolve.

In [1]:
import pandas as pd
import tensorflow as tf

ImportError: Traceback (most recent call last):
  File "/home/dhodun/.local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/dhodun/.local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/dhodun/.local/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [ ]:
print(tf.__version__)

## Tensorflow APIs
<img src="assets/TFHierarchy.png">

Tensorflow is a heirarchical framework. The further down the heirarchy you go, the more flexibility you have, but that more code you have to write. A best practice is start at the highest level of abstraction. Then if you need additional flexibility for some reason drop down one layer. 

For this tutorial we will be operating at the highest levels of Tensorflow abstraction, using Estimator and Experiments APIs.

## Steps

1. Load raw data

2. Write Tensorflow Code

 1. Define Feature Columns
 
 2. Define Estimator

 3. Define Input Function
 
 4. Define Serving Function

 5. Define Experiment

3. Package Code

4. Train

5. Inspect Results

6. Deploy Model

7. Get Predictions

### 1) Load Raw Data

This is a publically available dataset on housing prices in Boston area suburbs circa 1978. It is hosted in a Google Cloud Storage bucket.

For datasets too large to fit in memory you would read the data in batches. Tensorflow provides a queueing mechanism for this which is documented [here](https://www.tensorflow.org/programmers_guide/reading_data).

In our case the dataset is small enough to fit in memory so we will simply read it into a pandas dataframe.

In [ ]:
#downlad data from GCS and store as pandas dataframe 
data_train = pd.read_csv(
  filepath_or_buffer='https://storage.googleapis.com/vijay-public/boston_housing/housing_train.csv',
  names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","MEDV"])

data_test = pd.read_csv(
  filepath_or_buffer='https://storage.googleapis.com/vijay-public/boston_housing/housing_test.csv',
  names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","MEDV"])

In [ ]:
data_train.head()

#### Column Descriptions:

1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-retail business acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per $10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. MEDV: Median value of owner-occupied homes

### 2) Write Tensorflow Code

#### 2.A Define Feature Columns

Feature columns are your Estimator's data "interface." They tell the estimator in what format they should expect data and how to interpret it (is it one-hot? sparse? dense? categorical? conteninous?).  https://www.tensorflow.org/api_docs/python/tf/contrib/layers/feature_column




In [ ]:
FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM",
            "AGE", "DIS", "TAX", "PTRATIO"]
LABEL = "MEDV"

feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES] #list of Feature Columns

#### 2.B Define Estimator

An Estimator is what actually implements your training, eval and prediction loops. Every estimator has the following methods:

- fit() for training
- eval() for evaluation
- predict() for prediction
- export_savedmodel() for writing model state to disk

Tensorflow has several canned estimator that already implement these methods (DNNClassifier, LogisticClassifier etc..) or you can implement a custom estimator. Instructions on how to implement a custom estimator [here](https://www.tensorflow.org/extend/estimators) and see an example [here](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/timeseries/rnn_cloudmle.ipynb).

For simplicity we will use a canned estimator. To instantiate an estimator simply pass it what Feature Columns to expect.

In [ ]:
def generate_estimator(output_dir):
  return tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                            hidden_units=[10, 10],
                                            model_dir=output_dir)

#### 2.C Define Input Function

Now that you have an estimator and it knows what type of data to expect and how to intepret, you need to actually pass the data to it! This is the job of the input function. 

The input function returns a (features, label) tuple
- features: A dictionary. Each key is a feature column name and its value is the Tensor containing the data for that Feature
- label: A Tensor containing the label column

In [ ]:
def generate_input_fn(data_set):
    def input_fn():
      features = {k: tf.constant(data_set[k].values) for k in FEATURES}
      labels = tf.constant(data_set[LABEL].values)
      return features, labels
    return input_fn

#### 2.D Define Serving Input Function

To predict with the model, we need to define a serving input function which will be used to read inputs from a user at prediction time. If necessary transforms it also preforms transormations neccessary to get the user data into the format the estimator expects.

The serving input function resturs a (features, labels, inputs) tuple
- features: A dict of features to be passed to the Estimator
- label: Always 'None' for prediction
- inputs: A dictionary of inputs the predictions server should expect from the user

In [ ]:
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils

def serving_input_fn():
  feature_placeholders = {
      column.name: tf.placeholder(column.dtype, [None])
      for column in feature_cols
  }
  # DNNCombinedLinearClassifier expects rank 2 Tensors, but inputs should be
  # rank 1, so that we can provide scalars to the server
  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
  return input_fn_utils.InputFnOps(
    features,
    None,
    feature_placeholders
  )

#### 2.E Define Experiment

An experiment is ultimatley what you pass to learn_runner.run() for Cloud ML training. It encapsulated the Estimator (which in turn encapsulates the feature column definitions) and the Input Function.

Notice that to instantiate an Experiment you don't pass the Estimator or Input Functions directly. Instead you pass a function that *returns* an Estimator or Input Function. This is why we wrapped the Estimator and Input Function instantiations in generator functions during steps 3 and 4 respectively. It is also why we wrap the Experiment itself in a generator function, because a downstream function will expect it in this format.

In [ ]:
from tensorflow.contrib.learn.python.learn.utils import saved_model_export_utils

def generate_experiment_fn(output_dir):
  return tf.contrib.learn.Experiment(
    generate_estimator(output_dir),
    train_input_fn=generate_input_fn(data_train), 
    eval_input_fn=generate_input_fn(data_test), 
    export_strategies=[saved_model_export_utils.make_export_strategy(
            serving_input_fn,
            default_output_alternative_key=None,
        )],
    train_steps=3000,
    eval_steps=1
  )
  

### 3) Package Code

You've now written all the tensoflow code you need!

To make it compatible with Cloud ML Engine we'll combine the above tensorflow code into a single python file with two simple changes

1. Add some boilerplate code to parse the command line arguments required for gcloud.
2. Use the learn_runner.run() function to run the experiment

We also add an empty \__init__\.py file to the folder. This is just the python convention for identifying modules.

In [3]:
%%bash
mkdir trainer
touch trainer/__init__.py

mkdir: cannot create directory ‘trainer’: File exists


In [4]:
%%writefile trainer/task.py

import argparse
import pandas as pd
import tensorflow as tf
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import saved_model_export_utils
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils
from tensorflow.contrib.learn.python.learn.estimators import run_config as run_config_lib

tf.logging.set_verbosity(tf.logging.ERROR)

data_train = pd.read_csv(
  filepath_or_buffer='https://storage.googleapis.com/vijay-public/boston_housing/housing_train.csv',
  names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","MEDV"])

data_test = pd.read_csv(
  filepath_or_buffer='https://storage.googleapis.com/vijay-public/boston_housing/housing_test.csv',
  names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","MEDV"])

FEATURES = ["CRIM", "ZN", "INDUS", "NOX", "RM",
            "AGE", "DIS", "TAX", "PTRATIO"]
LABEL = "MEDV"

feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES] #list of Feature Columns

def generate_estimator(output_dir, hparams):
  return tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                            hidden_units=[10, 10],
                                            model_dir=output_dir)

def generate_input_fn(data_set):
    def input_fn():
      features = {k: tf.constant(data_set[k].values) for k in FEATURES}
      labels = tf.constant(data_set[LABEL].values)
      return features, labels
    return input_fn

def serving_input_fn():
  feature_placeholders = {
      column.name: tf.placeholder(column.dtype, [None])
      for column in feature_cols
  }
  # DNNCombinedLinearClassifier expects rank 2 Tensors, but inputs should be
  # rank 1, so that we can provide scalars to the server
  features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
  }
  return input_fn_utils.InputFnOps(
    features,
    None,
    feature_placeholders
  )

def generate_experiment_fn(output_dir, hparams):
  return tf.contrib.learn.Experiment(
    generate_estimator(output_dir, hparams=hparams),
    train_input_fn=generate_input_fn(data_train), 
    eval_input_fn=generate_input_fn(data_test), 
    export_strategies=[saved_model_export_utils.make_export_strategy(
            serving_input_fn,
            default_output_alternative_key=None,
        )],
    train_steps=train_steps,
    eval_steps=1
  )

######CLOUD ML ENGINE BOILERPLATE CODE BELOW######
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  parser.add_argument(
        '--job-dir',
        help='this model ignores this field, but it is required by gcloud',
        default='junk'
  )
  parser.add_argument(
        '--train_steps',
        help='this model ignores this field, but it is required by gcloud',
        default='100'
  )
  args = parser.parse_args()
  arguments = args.__dict__
  output_dir = arguments.pop('output_dir')
  train_steps = arguments.pop('train_steps')
  hparams = tf.contrib.training.HParams(train_steps=train_steps)

  learn_runner.run(generate_experiment_fn,
                   run_config=run_config_lib.RunConfig(model_dir=output_dir),
                   hparams=hparams)

Overwriting trainer/task.py


### 4) Train
Now that our code is packaged we can invoke it using the gcloud command line tool to run the training. 

Note: Since our dataset is so small and our model is simple the overhead of provisioning the cluster is longer than the actual training time. Accordingly you'll notice the single VM cloud training takes longer than the local training, and the distributed cloud training takes longer than single VM cloud. For larger datasets and more complex models this will reverse

#### Set Environment Vars
We'll create environment variables for our project name GCS Bucket and reference this in future commands.

In [5]:
GCS_BUCKET = 'gs://dhodun1-ml' #CHANGE THIS TO YOUR BUCKET
PROJECT = 'dhodun1' #CHANGE THIS TO YOUR PROJECT ID
REGION = 'us-central1' #OPTIONALLY CHANGE THIS

In [6]:
import os
os.environ['GCS_BUCKET'] = GCS_BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

#### Run local
It's a best practice to first run locally on a small dataset to check for errors. Note you can ignore the warnings in this case, as long as there are no errors.

In [7]:
%%bash
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=trainer \
   -- \
   --output_dir='./output'

Traceback (most recent call last):
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/dhodun/ml-teaching-examples/misc/trainer/task.py", line 96, in <module>
    hparams=hparams)
  File "/home/dhodun/.local/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/learn_runner.py", line 192, in run
    experiment = wrapped_experiment_fn(run_config=run_config, hparams=hparams)
  File "/home/dhodun/.local/lib/python2.7/site-packages/tensorflow/contrib/learn/python/learn/learn_runner.py", line 78, in wrapped_experiment_fn
    experiment = experiment_fn(run_config, hparams)
  File "/home/dhodun/ml-teaching-examples/misc/trainer/task.py", line 66, in generate_experiment_fn
    eval_steps=1
  File "/home/dhodun/.local/lib/python2.7/site-packages/tensorflow/python/util/deprecation.py", line 296, in new_func
    return func(

#### Run on cloud (1 cloud ML unit)

First we specify which GCP project to use.

In [3]:
%%bash
gcloud config set project $PROJECT

Updated property [core/project].


Then we specify which GCS bucket to write to and a job name.
Job names submitted to the ml engine must be project unique, so we append the system date/time. Update the cell below to point to a GCS bucket you own.

In [11]:
%%bash
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME/ \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output


jobId: housing_180223_013510
state: QUEUED


Job [housing_180223_013510] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe housing_180223_013510

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs housing_180223_013510


#### Run on cloud (10 cloud ML units)
Because we are using the TF Experiments interface, distributed computing just works! The only change we need to make to run in a distributed fashion is to add the [--scale-tier](https://cloud.google.com/ml/pricing#ml_training_units_by_scale_tier) argument. Cloud ML Engine then takes care of distributing the training across devices for you!


In [5]:
%%bash
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --scale-tier=STANDARD_1 \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: housing_180223_014959
state: QUEUED


Job [housing_180223_014959] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe housing_180223_014959

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs housing_180223_014959


#### Run on cloud GPU (3 cloud ML units)

Also works with GPUs!

"BASIC_GPU" corresponds to one Tesla K80 at the time of this writing, hardware subject to change. 1 GPU is charged as 3 cloud ML units.

In [ ]:
%%bash
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --scale-tier=BASIC_GPU \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

#### Run with Hyper-Parameter Tuning

We can also use the Hyper-Parameter tuning service in Cloud ML Engine to search the hyper-parameter space for best performance. You can run trials serially, or several trials at once.

In [13]:
%%writefile config_hypertune.yaml
trainingInput:
  scaleTier: BASIC
  hyperparameters:
    goal: MINIMIZE
    maxTrials: 2
    maxParallelTrials: 1
    params:
      - parameterName: train_steps
        type: INTEGER
        minValue: 3
        maxValue: 80
        scaleType: UNIT_LINEAR_SCALE

Overwriting config_hypertune.yaml


In [14]:
%%bash
JOBNAME=housing_$(date -u +%y%m%d_%H%M%S)

gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=./trainer \
   --job-dir=$GCS_BUCKET/$JOBNAME \
   --config=config_hypertune.yaml \
   -- \
   --output_dir=$GCS_BUCKET/$JOBNAME/output

jobId: housing_180223_024456
state: QUEUED


Job [housing_180223_024456] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe housing_180223_024456

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs housing_180223_024456


### 5) Inspect Results Using Tensorboard

Tensorboard is a utility that allows you to visualize your results.

Expand the 'loss' graph. What is your evaluation loss? This is squared error, so take the square root of it to get the average error in dollars. Does this seem like a reasonable margin of error for predicting a housing price?

In [4]:
from google.datalab.ml import TensorBoard
TensorBoard().start('output')

TensorBoard was started successfully with pid 25880. Click here to access it.

25880

Works with GCS URLs too

In [ ]:
!gsutil list $GCS_BUCKET | tail -1

In [ ]:
TensorBoard().start('gs://vijays-sandbox-ml/housing_170604_160854/output') #REPLACE with output from previous cell and append /output

Cleanup Tensorboard processes

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

### 6) Deploy Model For Predictions

Cloud ML Engine has a prediction service that will wrap our tensorflow model with a REST API and allow remote clients to get predictions.

You can deploy the model from the Google Cloud Console GUI, or you can use the gcloud command line tool. We will use the latter method.

In [ ]:
%%bash
MODEL_NAME="housing_prices"
MODEL_VERSION="v1"
MODEL_LOCATION="output/export/Servo/1501680438591" #REPLACE this with the location of your model

#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME} #Uncomment to overwrite existing version
#gcloud ml-engine models delete ${MODEL_NAME} #Uncomment to overwrite existing model
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --staging-bucket=$GCS_BUCKET

### 7) Get Predictions

There are two flavors of the ML Engine Prediction Service: Batch and online.

Online prediction is more appropriate for latency sensitive requests as results are returned quickly and synchronously. 

Batch prediction is more appropriate for large prediction requests that you only need to run a few times a day.

The prediction services expects prediction requests in standard JSON format so first we will create a JSON file with a couple of housing records.


In [ ]:
%%writefile records.json
{"CRIM": 0.00632,"ZN": 18.0,"INDUS": 2.31,"NOX": 0.538, "RM": 6.575, "AGE": 65.2, "DIS": 4.0900, "TAX": 296.0, "PTRATIO": 15.3}
{"CRIM": 0.00332,"ZN": 0.0,"INDUS": 2.31,"NOX": 0.437, "RM": 7.7, "AGE": 40.0, "DIS": 5.0900, "TAX": 250.0, "PTRATIO": 17.3}

Now we will pass this file to the prediction service using the gcloud command line tool. Results are returned immediatley!

In [ ]:
!gcloud ml-engine predict --model housing_prices --json-instances records.json

### Conclusion

#### What we covered
1. How to use Tensorflow's high level Estimator API
2. How to deploy tensorflow code for distributed training in the cloud
3. How to evaluate results using TensorBoard
4. How deploy the resulting model to the cloud for online prediction

#### What we didn't cover
1. How to leverage larger than memory datasets using Tensorflow's queueing system
2. How to create synthetic features from our raw data to aid learning (Feature Engineering)
3. How to improve model performance by finding the ideal hyperparameters using Cloud ML Engine's [HyperTune](https://cloud.google.com/ml-engine/docs/how-tos/using-hyperparameter-tuning) feature

This lab is a great start, but adding in the above concepts is critical in getting your models to production ready quality. These concepts are covered in Google's 1-week on-demand Tensorflow + Cloud ML course: https://www.coursera.org/learn/serverless-machine-learning-gcp